In [ ]:
from byzerllm.apps.byzer_storage.simple_api import ByzerStorage, DataType, FieldOption,SortOption

# 初始化 EasyStorage
storage = ByzerStorage("byzerai_store", "projects", "auto-coder")
_ = (
    storage.schema_builder()
    ## 主键，字符串
    .add_field("_id", DataType.STRING)
    ## 可以作为where条件过滤的字段
    .add_field("name", DataType.STRING)
    ## 可以作为搜索字段
    .add_field("content", DataType.STRING, [FieldOption.ANALYZE])
    ## 不需要索引的字段，会原模原样存储
    .add_field("raw_content", DataType.STRING, [FieldOption.NO_INDEX])    
    ## 数组字段
    .add_array_field("summary", DataType.FLOAT)    
    ## 需要排序的字段
    .add_field("created_time", DataType.LONG, [FieldOption.SORT])    
    .execute()
)

## st(
# field(_id,long),
# field(name,string),
# field(content,string,analyze),
# field(raw_content,string,no_index),
# field(summary,array(float)),
# field(created_time,long,sort)
# )

# 准备数据
data = [
    {"_id": "1", "name": "Hello", "content": "Hello, world!", "raw_content": "Hello, world!", "summary": "hello world", "created_time": 1612137600},
    {"_id": "2", "name": "Byzer", "content": "Byzer, world!", "raw_content": "Byzer, world!", "summary": "byzer", "created_time": 1612137601},
    {"_id": "3", "name": "AI", "content": "AI, world!", "raw_content": "AI, world!", "summary": "AI", "created_time": 16121376002},
    {"_id": "4", "name": "ByzerAI", "content": "ByzerAI, world!", "raw_content": "ByzerAI, world!", "summary": "ByzerAi", "created_time": 16121376003},
]

# 写入Storage
storage.write_builder().add_items(data,vector_fields=["summary"],search_fields=["content"]).execute()

# commit 写入
storage.commit()

# 搜索
query = storage.query_builder()
query.set_vector_query("ByzerAi",fields=["summary"])
results = query.set_search_query("Hello",fields=["content"]).execute()
print(results)

## 过滤之后再做搜索
query = storage.query_builder()
query.and_filter().add_condition("name","AI").build()
query.set_vector_query("ByzerAi",fields="summary")
results = query.set_search_query("Hello",fields=["content"]).execute()
print(results)

## 过滤再做排序
query = storage.query_builder()
query.and_filter().add_condition("name","AI").build().sort("created_time",SortOption.DESC)
results = query.execute()
print(results)

## 根据ID做删除
storage.delete_by_ids(["3"])

query = storage.query_builder()
query.and_filter().add_condition("name","AI").build()
results = query.execute()
print("====")
print(results)

## 删除表
storage.drop_table()

In [ ]:
from byzerllm.apps.byzer_storage.simple_api import ByzerStorage, DataType, FieldOption,SortOption
storage = ByzerStorage("byzerai_store", "my_database1", "my_table4s")
## 根据过滤条件删除数据
storage.query_builder().and_filter().add_condition("name","AI").build().delete()
# storage.delete_by_ids(["3"])

query = storage.query_builder()
query.and_filter().add_condition("name","AI").build()
results = query.execute()
print("====")
print(results)

In [ ]:
from byzerllm.apps.byzer_storage.simple_api import ByzerStorage, DataType, FieldOption,SortOption

# 初始化 EasyStorage
storage = ByzerStorage("byzerai_store", "memory", "memory")
_ = (
    storage.schema_builder()
    ## 主键，字符串
    .add_field("_id", DataType.STRING)
    ## 可以作为where条件过滤的字段
    .add_field("name", DataType.STRING)
    ## 可以作为搜索字段
    .add_field("content", DataType.STRING, [FieldOption.ANALYZE])
    ## 不需要索引的字段，会原模原样存储
    .add_field("raw_content", DataType.STRING, [FieldOption.NO_INDEX])    
    ## 数组字段
    .add_array_field("summary", DataType.FLOAT)    
    ## 需要排序的字段
    .add_field("created_time", DataType.LONG, [FieldOption.SORT])    
    .execute()
)



In [ ]:
import time

int(time.time())


In [ ]:
import time
import uuid
def memorize(name:str,s:str):
    data = [
        {"_id": str(uuid.uuid4()), "name": name, "content": s, "raw_content": s, "summary": s, "created_time": int(time.time())},    
    ]
    storage.write_builder().add_items(data,vector_fields=["summary"],search_fields=["content"]).execute()

def recall(name:str,query:str):
    searcher = storage.query_builder()
    searcher.and_filter().add_condition("name",name).build()
    searcher.set_vector_query(query,fields="summary")
    results = searcher.set_search_query(query,fields=["content"]).execute() 
    return results

In [ ]:
storage.delete_by_ids(["1"])

In [ ]:
memorize("william","William的生日是2月1号")

In [ ]:

recall("william","William生日是几号")

In [ ]:
import byzerllm
import json

llm = byzerllm.ByzerLLM.from_default_model("deepseek_chat")

@byzerllm.prompt(llm=llm)
def ask_about_william(query:str)->str:
    '''
    根据下面的信息回答用户的问题。

    {{ context }}

    用户的问题：

    {{ query }}
    '''
    return {
        "context": json.dumps(recall("william",query),ensure_ascii=False)
    }

ask_about_william("William的生日是几号")

In [ ]:
import byzerllm
llm = byzerllm.ByzerLLM.from_default_model("deepseek_chat")
llm.setup_default_emb_model_name("emb")
llm.emb_query("你好")

In [2]:
from byzerllm.apps.byzer_storage.simple_api import ByzerStorage, DataType, FieldOption,SortOption
storage = ByzerStorage("byzerai_store", "memory", "memory")
storage.emb("你好")

2024-08-05 14:20:26.487 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-08-05 14:20:26,644	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-08-05 14:20:26,646	INFO worker.py:1582 -- Calling ray.init() again after it has already been called.


21


AssertionError: 